In [121]:
import pandas as pd
import geopandas as gpd
import maup
import time
from maup import smart_repair
from gerrychain import Graph
import os

maup.progress.enabled = True

In [12]:
import warnings
warnings.filterwarnings("ignore")

In [13]:
# parameters
# state = Connecticut
state_ab = "ct"

In [40]:
data_folder = state_ab + "_data/"
population1_data = "./{}{}_pl2020_b/{}_pl2020_b.shp".format(data_folder, state_ab, state_ab)
population2_data = "./{}{}_pl2020_b/{}_pl2020_b.shp".format(data_folder, state_ab, state_ab)
vap_data = "./{}{}_pl2020_b/{}_pl2020_b.shp".format(data_folder, state_ab, state_ab)
vest20_data = "./{}{}_vest_20/{}_vest_20.shp".format(data_folder, state_ab, state_ab)
vest18_data = "./{}{}_vest_18/{}_vest_18.shp".format(data_folder, state_ab, state_ab)
vest16_data = "./{}{}_vest_16/{}_vest_16.shp".format(data_folder, state_ab, state_ab)
cd_data = "./{}{}_cong_adopted_2022/Districts_1 2022-02-14.shp".format(data_folder, state_ab)
send_data = "./{}{}_sldu_2021/{}_sldu_2021.shp".format(data_folder, state_ab, state_ab)
hdist_data = "./{}{}_sldl_2021/{}_sldl_2021.shp".format(data_folder, state_ab, state_ab)

In [15]:
def do_smart_repair(df):
    df = df.to_crs(df.estimate_utm_crs())
    df = smart_repair(df)
    if maup.doctor(df):
        print('smart_repair successful')
        df = df.to_crs('EPSG:4269')
    else:
        print('smart_repair failed')
    return df

In [50]:
def add_district(dist_df, dist_name, election_df, col_name):
    # check if it needs to be smart_repair
    if maup.doctor(dist_df) != True:
        dist_df = do_smart_repair(dist_df)
    
    election_df = gpd.GeoDataFrame(election_df, crs="EPSG:4269")
    
    # assigne the pricincts
    precincts_to_district_assignment = maup.assign(election_df.geometry, dist_df.geometry)
    election_df[dist_name] = precincts_to_district_assignment
    for precinct_index in range(len(election_df)):
        election_df.at[precinct_index, dist_name] = dist_df.at[election_df.at[precinct_index, dist_name], col_name]
    
    return election_df

In [17]:
def rename(original, year):
    party = original[6]
    if party == 'R' or party == 'D':
        return original[3:6] + year + original[6]
    else:
        return original[3:6] + year + 'O'

In [86]:
def add_vest_data(vest_data, df, year, block_df, start_col = 5):
    vest = gpd.read_file(vest_data)
    
    if maup.doctor(vest) != True:
        vest = do_smart_repair(vest)
        
    original_col = vest.columns[start_col:-1]
    new_col = [rename(i, year) for i in original_col]
    rename_dict = dict(zip(original_col, new_col))
    vest = vest.rename(columns=rename_dict)
    vest = vest.groupby(level=0, axis=1).sum()
    col_name = list(set(new_col))
    col_name.sort()
    
    # assign pricinct to block
    vest = gpd.GeoDataFrame(vest, crs="EPSG:4269")
    vest_to_block_assginment = maup.assign(vest.geometry, block_df.geometry)
    block = block_df[['geometry']]
    block[col_name] = vest[col_name].groupby(vest_to_block_assginment).sum()
    
    # assign block to vest
    block = gpd.GeoDataFrame(block, crs="EPSG:4269")
    df = gpd.GeoDataFrame(df, crs="EPSG:4269")
    block_to_pricinct_assginment = maup.assign(block.geometry, df.geometry)
    df[col_name] = block[col_name].groupby(block_to_pricinct_assginment).sum()
    df = df.groupby(level=0, axis=1).sum()
    
    return df

In [21]:
population_df = gpd.read_file(population1_data)

In [22]:
maup.doctor(population_df)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 49926/49926 [00:44<00:00, 1130.16it/s]


True

In [27]:
population_df['H_WHITE'] = population_df.apply(lambda t: t['P0010003'] - t['P0020005'], 1)
population_df['H_BLACK'] = population_df.apply(lambda t: t['P0010004'] - t['P0020006'], 1)
population_df['H_AMIN'] = population_df.apply(lambda t: t['P0010005'] - t['P0020007'], 1)
population_df['H_ASIAN'] = population_df.apply(lambda t: t['P0010006'] - t['P0020008'], 1)
population_df['H_NHPI'] = population_df.apply(lambda t: t['P0010007'] - t['P0020009'], 1)
population_df['H_OTHER'] = population_df.apply(lambda t: t['P0010008'] - t['P0020010'], 1)
population_df['H_2MORE'] = population_df.apply(lambda t: t['P0010009'] - t['P0020011'], 1)

In [71]:
vest20 = gpd.read_file(vest20_data)

In [66]:
maup.doctor(vest20)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 741/741 [00:01<00:00, 575.53it/s]


There are 9 overlaps.


False

In [67]:
vest20 = do_smart_repair(vest20)

Snapping all geometries to a grid with precision 10^( -5 ) to avoid GEOS errors.
Identifying overlaps...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 805/805 [00:00<00:00, 2946.46it/s]


Resolving overlaps...
Assigning order 2 pieces...
Assigning order 3 pieces...
Filling gaps...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 741/741 [00:01<00:00, 652.34it/s]


smart_repair successful


In [68]:
maup.doctor(vest20)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 741/741 [00:01<00:00, 609.16it/s]


True

In [70]:
vest20.columns

Index(['STATEFP20', 'COUNTYFP20', 'NAME20', 'G20PREDBID', 'G20PRERTRU',
       'G20PRELJOR', 'G20PREGHAW', 'G20PREOWRI', 'geometry'],
      dtype='object')

In [73]:
original_col = vest20.columns[3:-1]
new_col = [rename(i, '20') for i in original_col]
rename_dict = dict(zip(original_col, new_col))
vest20 = vest20.rename(columns=rename_dict)
vest20 = vest20.groupby(level=0, axis=1).sum()
vest20 = gpd.GeoDataFrame(vest20, crs="EPSG:4269")

In [74]:
# vap and population have the same GEOID20
blocks_to_precincts_assignment = maup.assign(population_df.geometry, vest20.geometry)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 741/741 [00:00<00:00, 1390.47it/s]


In [75]:
pop_column_names = ['P0020001', 'P0020002', 'P0020005', 'P0020006', 'P0020007', 'P0020008', 'P0020009', 'P0020010', 'P0020011', 
                    'H_WHITE', 'H_BLACK', 'H_AMIN', 'H_ASIAN', 'H_NHPI', 'H_OTHER', 'H_2MORE']
vap_column_names = ['P0040001', 'P0040002', 'P0040005', 'P0040006', 'P0040007', 'P0040008', 'P0040009', 'P0040010', 'P0040011']
pop_col = pop_column_names + vap_column_names

In [76]:
vest20[pop_col] = population_df[pop_col].groupby(blocks_to_precincts_assignment).sum()

In [77]:
cong_df = gpd.read_file(cd_data)
send = gpd.read_file(send_data)
hdist = gpd.read_file(hdist_data)

In [78]:
rename_dict = {'P0020001': 'TOTPOP', 'P0020002': 'HISP', 'P0020005': 'NH_WHITE', 'P0020006': 'NH_BLACK', 'P0020007': 'NH_AMIN',
                    'P0020008': 'NH_ASIAN', 'P0020009': 'NH_NHPI', 'P0020010': 'NH_OTHER', 'P0020011': 'NH_2MORE',
                    'P0040001': 'VAP', 'P0040002': 'HVAP', 'P0040005': 'WVAP', 'P0040006': 'BVAP', 'P0040007': 'AMINVAP',
                                        'P0040008': 'ASIANVAP', 'P0040009': 'NHPIVAP', 'P0040010': 'OTHERVAP', 'P0040011': '2MOREVAP'}

In [79]:
vest20.rename(columns=rename_dict, inplace = True)

In [80]:
vest20.columns

Index(['COUNTYFP20', 'NAME20', 'PRE20D', 'PRE20O', 'PRE20R', 'STATEFP20',
       'geometry', 'TOTPOP', 'HISP', 'NH_WHITE', 'NH_BLACK', 'NH_AMIN',
       'NH_ASIAN', 'NH_NHPI', 'NH_OTHER', 'NH_2MORE', 'H_WHITE', 'H_BLACK',
       'H_AMIN', 'H_ASIAN', 'H_NHPI', 'H_OTHER', 'H_2MORE', 'VAP', 'HVAP',
       'WVAP', 'BVAP', 'AMINVAP', 'ASIANVAP', 'NHPIVAP', 'OTHERVAP',
       '2MOREVAP'],
      dtype='object')

In [90]:
election_df = gpd.GeoDataFrame(vest20, crs="EPSG:4269")

In [91]:
election_df = add_vest_data(vest18_data, election_df, '18', population_df, 3)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 742/742 [00:01<00:00, 652.30it/s]


There are 9 overlaps.
Snapping all geometries to a grid with precision 10^( -5 ) to avoid GEOS errors.
Identifying overlaps...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 806/806 [00:00<00:00, 2950.47it/s]


Resolving overlaps...
Assigning order 2 pieces...
Assigning order 3 pieces...
Filling gaps...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 742/742 [00:01<00:00, 613.59it/s]


smart_repair successful


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 741/741 [00:00<00:00, 1392.12it/s]


In [92]:
election_df = add_vest_data(vest16_data, election_df, '16', population_df, 3)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 743/743 [00:01<00:00, 619.59it/s]


There are 9 overlaps.
Snapping all geometries to a grid with precision 10^( -5 ) to avoid GEOS errors.
Identifying overlaps...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 807/807 [00:00<00:00, 2928.83it/s]


Resolving overlaps...
Assigning order 2 pieces...
Assigning order 3 pieces...
Filling gaps...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 743/743 [00:01<00:00, 605.10it/s]


smart_repair successful


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 741/741 [00:00<00:00, 1388.95it/s]


In [93]:
election_df = add_district(cong_df, "CD", election_df, "ID")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00,  9.36it/s]


In [94]:
election_df = add_district(send, "SEND", election_df, "DISTRICTN")

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 36/36 [00:00<00:00, 95.48it/s]


There are 3 overlaps.
There are 35 holes.
Snapping all geometries to a grid with precision 10^( -5 ) to avoid GEOS errors.
Identifying overlaps...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [00:00<00:00, 1730.49it/s]


Resolving overlaps...
Assigning order 2 pieces...
Filling gaps...


Gaps to simplify: 0it [00:00, ?it/s]
Gaps to fill: 0it [00:00, ?it/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 36/36 [00:00<00:00, 198.92it/s]


smart_repair successful


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 36/36 [00:01<00:00, 25.48it/s]


In [95]:
election_df = add_district(hdist, "HDIST", election_df, "DISTRICTN")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 151/151 [00:00<00:00, 314.40it/s]


There are 8 overlaps.
There are 143 holes.
Snapping all geometries to a grid with precision 10^( -5 ) to avoid GEOS errors.
Identifying overlaps...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 156/156 [00:00<00:00, 2404.00it/s]


Resolving overlaps...
Assigning order 2 pieces...
Filling gaps...


Gaps to simplify: 0it [00:00, ?it/s]
Gaps to fill: 0it [00:00, ?it/s]
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 151/151 [00:00<00:00, 391.11it/s]


smart_repair successful


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 151/151 [00:01<00:00, 89.14it/s]


In [105]:
list(election_df.columns)

['2MOREVAP',
 'AMINVAP',
 'ASIANVAP',
 'ATG18D',
 'ATG18O',
 'ATG18R',
 'BVAP',
 'COM18D',
 'COM18O',
 'COM18R',
 'COUNTYFP20',
 'GOV18D',
 'GOV18O',
 'GOV18R',
 'HISP',
 'HVAP',
 'H_2MORE',
 'H_AMIN',
 'H_ASIAN',
 'H_BLACK',
 'H_NHPI',
 'H_OTHER',
 'H_WHITE',
 'NAME20',
 'NHPIVAP',
 'NH_2MORE',
 'NH_AMIN',
 'NH_ASIAN',
 'NH_BLACK',
 'NH_NHPI',
 'NH_OTHER',
 'NH_WHITE',
 'OTHERVAP',
 'PRE16D',
 'PRE16O',
 'PRE16R',
 'PRE20D',
 'PRE20O',
 'PRE20R',
 'SOS18D',
 'SOS18O',
 'SOS18R',
 'STATEFP20',
 'TOTPOP',
 'TRE18D',
 'TRE18O',
 'TRE18R',
 'USS16D',
 'USS16O',
 'USS16R',
 'USS18D',
 'USS18O',
 'USS18R',
 'VAP',
 'WVAP',
 'geometry',
 'CD',
 'SEND',
 'HDIST']

In [113]:
fixed_columns = [
    'STATEFP20',
    'COUNTYFP20',
    'NAME20',
    # 'STATEFP',
    # 'COUNTYFP',
    # 'VTDST',
    # 'PRECINCT',
    # 'NAME',
    'CD',
    'SEND',
    'HDIST',
    'TOTPOP',
    'NH_2MORE',
    'NH_AMIN',
    'NH_ASIAN',
    'NH_BLACK',
    'NH_NHPI',
    'NH_OTHER',
    'NH_WHITE',
    'HISP',
    'H_AMIN',
    'H_ASIAN',
    'H_BLACK',
    'H_NHPI',
    'H_OTHER',
    'H_WHITE',
    'H_2MORE',
    'VAP',
    'HVAP',
    'WVAP',
    'BVAP',
    'AMINVAP',
    'ASIANVAP',
    'NHPIVAP',
    'OTHERVAP',
    '2MOREVAP']

In [114]:
election_columns = [col for col in election_df.columns if col not in fixed_columns]

In [115]:
final_col = fixed_columns + election_columns

In [116]:
election_df = election_df[final_col]

In [ ]:
maup.doctor(election_df)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 741/741 [00:01<00:00, 577.58it/s]


There are 9 overlaps.


False

In [118]:
election_df = do_smart_repair(election_df)

Snapping all geometries to a grid with precision 10^( -5 ) to avoid GEOS errors.
Identifying overlaps...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 805/805 [00:00<00:00, 2971.93it/s]


Resolving overlaps...
Assigning order 2 pieces...
Assigning order 3 pieces...
Filling gaps...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 741/741 [00:01<00:00, 626.28it/s]


smart_repair successful


In [119]:
maup.doctor(election_df)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 741/741 [00:01<00:00, 608.65it/s]


True

In [122]:
os.makedirs("./{}".format(state_ab))

election_df.to_file("./{}/{}.shp".format(state_ab, state_ab))
election_df.to_file('./{}/{}.geojson'.format(state_ab, state_ab), driver='GeoJSON')

#Only do once to build json and read from file when generating ensembles
graph = Graph.from_file("./{}/{}.shp".format(state_ab, state_ab), ignore_errors=True)
graph.to_json("./{}/{}.json".format(state_ab, state_ab))